# PUNTO 3 - ARQUITECTURA 2


# Descarga de librerias necesarias

In [1]:
# !pip3 install tensorflow

## Importar librerias necesarias

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from keras.layers import Embedding, Flatten, Dense, Dropout
import os
from gensim.utils import simple_preprocess
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Embedding, Flatten
from keras.callbacks import EarlyStopping




## PREPROCESAMIENTO

## Carga de datos

In [3]:


books_folder = './books/'


book_files = [f for f in os.listdir(books_folder) if f.endswith('.txt')]

names = []
books_texts = []
for book_file in book_files:
    with open(os.path.join(books_folder, book_file), 'r', encoding='utf-8') as file:
        books_texts.append(file.read())
        names.append(book_file.split('_')[0])

print(f'Se encontraron {len(book_files)} libros en la carpeta /books')
processed_books = [" ".join(simple_preprocess(text)) for text in books_texts]



Se encontraron 9 libros en la carpeta /books


## Dividir textos en fragmentos con su respectiva etiqueta

In [4]:
def split_into_fragments(text, fragment_size=150):
    words = text.split()
    fragments = [words[i:i + fragment_size] for i in range(0, len(words), fragment_size)]
    return [' '.join(fragment) for fragment in fragments]


fragment_size = 150  
fragmented_books = []
fragment_labels = []

for i, book_text in enumerate(books_texts):
    fragments = split_into_fragments(book_text, fragment_size=fragment_size)
    fragmented_books.extend(fragments)
    fragment_labels.extend([names[i]] * len(fragments))  


## Revision de fragmentos por etiqueta

In [5]:
print(len(fragment_labels))
dic = {}
for i in fragment_labels:
    dic[i] = dic.get(i, 0)+ 1

print(dic)

7553
{'tolstoy': 4544, 'forster': 1520, 'vonarnin': 1489}


## Tokenizacion

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(fragmented_books)
X = tokenizer.texts_to_sequences(fragmented_books)

X = pad_sequences(X, maxlen=fragment_size)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(fragment_labels)


## Division entre test y train

In [7]:
!pip install imbalanced-learn


Defaulting to user installation because normal site-packages is not writeable


In [8]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)


In [9]:
vocab_size = len(tokenizer.word_index) + 1  


## Creacion de la matriz de embeddings pre entrenados

## Embeddings de tamaño 300

In [10]:
from gensim.models import Word2Vec  
embedding_model_3 = Word2Vec.load('Books_300_EMF.model')
vocab_size = len(tokenizer.word_index) + 1  
embedding_dim_3 = embedding_model_3.vector_size  
embedding_matrix_3 = np.zeros((vocab_size, embedding_dim_3))

In [11]:
for word, idx in tokenizer.word_index.items():
    if word in embedding_model_3.wv:
        embedding_matrix_3[idx] = embedding_model_3.wv[word]
    else:
        embedding_matrix_3[idx] = np.zeros(embedding_dim_3)

## Embedding de tamaño 200


In [12]:
from gensim.models import Word2Vec  
embedding_model_2 = Word2Vec.load('Books_200_EMF.model')
vocab_size = len(tokenizer.word_index) + 1  
embedding_dim_2 = embedding_model_2.vector_size  
embedding_matrix_2 = np.zeros((vocab_size, embedding_dim_2))

In [13]:
for word, idx in tokenizer.word_index.items():
    if word in embedding_model_2.wv:
        embedding_matrix_2[idx] = embedding_model_2.wv[word]
    else:
        embedding_matrix_2[idx] = np.zeros(embedding_dim_2)


# Embeddings de tamaño 100

In [14]:
from gensim.models import Word2Vec  
embedding_model = Word2Vec.load('Books_100_EMF.model')
vocab_size = len(tokenizer.word_index) + 1  
embedding_dim = embedding_model.vector_size  
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [15]:
for word, idx in tokenizer.word_index.items():
    if word in embedding_model.wv:
        embedding_matrix[idx] = embedding_model.wv[word]
    else:\
        embedding_matrix[idx] = np.zeros(embedding_dim)


## Creacion de la red feed-forward: Arquitectura 1

### Embeddings tamaño 300

In [16]:



model_3 = Sequential()

model_3.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim_3, 
                    weights=[embedding_matrix_3],  
                    input_length=fragment_size,  
                    trainable=False))  

model_3.add(Flatten())

model_3.add(Dense(256, activation='relu'))
model_3.add(Dropout(0.5))

model_3.add(Dense(128, activation='relu'))
model_3.add(Dropout(0.5))

model_3.add(Dense(64, activation='relu'))

model_3.add(Dense(3, activation='softmax'))

model_3.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          10523400  
                                                                 
 flatten (Flatten)           (None, 45000)             0         
                                                                 
 dense (Dense)               (None, 256)               11520256  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [17]:
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Embeddings tamaño 200

In [18]:
set(names)

{'forster', 'tolstoy', 'vonarnin'}

In [19]:
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{0: 3635, 1: 3635, 2: 3635}


In [20]:
model_2 = Sequential()

model_2.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim_2, 
                    weights=[embedding_matrix_2],  
                    input_length=fragment_size,  
                    trainable=False))  

model_2.add(Flatten())

model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(64, activation='relu'))

model_2.add(Dense(3, activation='softmax'))

model_2.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 200)          7015600   
                                                                 
 flatten_1 (Flatten)         (None, 30000)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               7680256   
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [21]:
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Embeddings tamaño 100

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout

model = Sequential()

model.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim, 
                    weights=[embedding_matrix],  
                    input_length=fragment_size,  
                    trainable=False))  

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dense(3, activation='softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 100)          3507800   
                                                                 
 flatten_2 (Flatten)         (None, 15000)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               3840256   
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)               

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Entrenamiento de la red

### Embeddings tamaño 100

In [24]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
341/341 [==============================] - 4s 11ms/step - loss: 1.1684 - accuracy: 0.3394 - val_loss: 1.0930 - val_accuracy: 0.3762
Epoch 2/100
341/341 [==============================] - 4s 10ms/step - loss: 1.1026 - accuracy: 0.3541 - val_loss: 1.0915 - val_accuracy: 0.3715
Epoch 3/100
341/341 [==============================] - 3s 10ms/step - loss: 1.0854 - accuracy: 0.3756 - val_loss: 1.0740 - val_accuracy: 0.4067
Epoch 4/100
341/341 [==============================] - 4s 11ms/step - loss: 1.0705 - accuracy: 0.3970 - val_loss: 1.0520 - val_accuracy: 0.4411
Epoch 5/100
341/341 [==============================] - 4s 11ms/step - loss: 1.0484 - accuracy: 0.4171 - val_loss: 1.0294 - val_accuracy: 0.4507
Epoch 6/100
341/341 [==============================] - 3s 10ms/step - loss: 1.0152 - accuracy: 0.4503 - val_loss: 0.9961 - val_accuracy: 0.5064
Epoch 7/100
341/341 [==============================] - 3s 10ms/step - loss: 0.9775 - accuracy: 0.4777 - val_loss: 0.9708 - val_accuracy:

### Embeddings tamaño 200

In [25]:
early_stopping_2 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
341/341 [==============================] - 7s 20ms/step - loss: 1.2006 - accuracy: 0.3375 - val_loss: 1.0952 - val_accuracy: 0.3352
Epoch 2/100
341/341 [==============================] - 7s 21ms/step - loss: 1.0986 - accuracy: 0.3486 - val_loss: 1.0874 - val_accuracy: 0.3891
Epoch 3/100
341/341 [==============================] - 8s 23ms/step - loss: 1.0882 - accuracy: 0.3763 - val_loss: 1.0706 - val_accuracy: 0.3975
Epoch 4/100
341/341 [==============================] - 7s 21ms/step - loss: 1.0735 - accuracy: 0.3875 - val_loss: 1.0421 - val_accuracy: 0.4690
Epoch 5/100
341/341 [==============================] - 7s 21ms/step - loss: 1.0547 - accuracy: 0.4138 - val_loss: 1.0288 - val_accuracy: 0.4683
Epoch 6/100
341/341 [==============================] - 6s 18ms/step - loss: 1.0410 - accuracy: 0.4293 - val_loss: 1.0158 - val_accuracy: 0.4723
Epoch 7/100
341/341 [==============================] - 7s 20ms/step - loss: 1.0262 - accuracy: 0.4359 - val_loss: 1.0009 - val_accuracy:

In [26]:
loss, accuracy = model_2.evaluate(X_test, y_test)

86/86 [==============================] - 1s 5ms/step - loss: 0.6937 - accuracy: 0.7180


### Embeddings tamaño 300

In [27]:
early_stopping_3 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_3.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
341/341 [==============================] - 10s 28ms/step - loss: 1.2083 - accuracy: 0.3288 - val_loss: 1.0958 - val_accuracy: 0.3553
Epoch 2/100
341/341 [==============================] - 11s 32ms/step - loss: 1.0993 - accuracy: 0.3427 - val_loss: 1.0879 - val_accuracy: 0.3682
Epoch 3/100
341/341 [==============================] - 10s 29ms/step - loss: 1.0905 - accuracy: 0.3543 - val_loss: 1.0711 - val_accuracy: 0.3905
Epoch 4/100
341/341 [==============================] - 13s 38ms/step - loss: 1.0730 - accuracy: 0.3910 - val_loss: 1.0639 - val_accuracy: 0.4162
Epoch 5/100
341/341 [==============================] - 11s 33ms/step - loss: 1.0588 - accuracy: 0.4062 - val_loss: 1.0411 - val_accuracy: 0.4470
Epoch 6/100
341/341 [==============================] - 12s 35ms/step - loss: 1.0376 - accuracy: 0.4268 - val_loss: 1.0093 - val_accuracy: 0.4862
Epoch 7/100
341/341 [==============================] - 11s 33ms/step - loss: 1.0233 - accuracy: 0.4397 - val_loss: 1.0057 - val_ac

In [28]:
loss, accuracy = model_3.evaluate(X_test, y_test)

86/86 [==============================] - 1s 7ms/step - loss: 0.6606 - accuracy: 0.7429


# Resultados  y metricas

### Embeddings tamaño 100

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 100)          3507800   
                                                                 
 flatten_2 (Flatten)         (None, 15000)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               3840256   
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)               

In [30]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  

86/86 [==============================] - 1s 4ms/step


In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.84      0.71      0.77       909
           1       0.58      0.76      0.66       909
           2       0.83      0.69      0.75       909

    accuracy                           0.72      2727
   macro avg       0.75      0.72      0.73      2727
weighted avg       0.75      0.72      0.73      2727



### Embeddings tamaño 200

In [32]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 200)          7015600   
                                                                 
 flatten_1 (Flatten)         (None, 30000)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               7680256   
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [33]:
y_pred = model_2.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1) 

86/86 [==============================] - 1s 10ms/step


In [34]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.86      0.66      0.74       909
           1       0.58      0.76      0.66       909
           2       0.80      0.74      0.77       909

    accuracy                           0.72      2727
   macro avg       0.74      0.72      0.72      2727
weighted avg       0.74      0.72      0.72      2727



### Embeddings tamaño 300

In [35]:
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 300)          10523400  
                                                                 
 flatten (Flatten)           (None, 45000)             0         
                                                                 
 dense (Dense)               (None, 256)               11520256  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [36]:
y_pred = model_3.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1) 

86/86 [==============================] - 1s 9ms/step


In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_classes)) 

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       909
           1       0.61      0.77      0.68       909
           2       0.81      0.82      0.81       909

    accuracy                           0.74      2727
   macro avg       0.77      0.74      0.75      2727
weighted avg       0.77      0.74      0.75      2727

